In [2]:
from atproto import Client, client_utils
from dotenv import load_dotenv
import sqlite3
import time 
import os

load_dotenv()

username = os.getenv('username')
password = os.getenv('password')

client = Client()
client.login(username, password)

ValueError: Either session_string or login and password should be provided.

In [7]:
# Set up SQLite database
conn = sqlite3.connect('bsky_posts.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        author TEXT,
        text TEXT
    )
''')
conn.commit()

print("Database setup complete. Starting feed...")

Database setup complete. Starting feed...


In [8]:
# Keep track of the most recent post we've seen
last_seen_post = None

while True:
    try:
        # Get the timeline
        timeline = client.get_timeline(limit=50)  # Adjust limit as needed

        # Iterate through posts, newest first
        for post in reversed(timeline.feed):
            # If we've seen this post before, skip to the next one
            if last_seen_post and post.post.cid == last_seen_post:
                break
            
            # Insert post into database
            cursor.execute('''
                INSERT INTO posts (author, text) VALUES (?, ?)
            ''', (post.post.author.handle, post.post.record.text))
            
            # # Print post details
            # print(f"Author: {post.post.author.handle}")
            # print(f"Text: {post.post.record.text}")
            # print("---")

        # Commit the changes to the database
        conn.commit()

        # Update the last seen post
        if timeline.feed:
            last_seen_post = timeline.feed[0].post.cid

        # Wait for a bit before checking again
        time.sleep(60)  # Wait for 60 seconds
        
    except KeyboardInterrupt:
        print("\nExiting the feed loop...")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Waiting before trying again...")
        time.sleep(300)  # Wait for 5 minutes before trying again

# Close the database connection
conn.close()
print("Database connection closed. Exiting program.")

# read the data 

In [5]:
# Connect to the database
conn = sqlite3.connect('bsky_posts.db')
cursor = conn.cursor()

# Read all posts from the database
cursor.execute('SELECT * FROM posts')
posts = cursor.fetchall()

In [6]:
# Display the posts
print(f"Total posts: {len(posts)}")
print("\nSample of posts:")
for post in posts[:5]:  # Show first 5 posts
    post_id, author, text = post
    print(f"\nPost ID: {post_id}")
    print(f"Author: {author}")
    print(f"Text: {text}")
    print("-" * 50)

# Close the connection
conn.close()

Total posts: 3884

Sample of posts:

Post ID: 1
Author: avt.im
Text: A part of those dynamics is that more and more papers are getting submitted to ML venues, while other venues grow slower if at all.

Even people who repeatedly complain about ML venues still submit there, even work they could instead send to various journals. This isn’t by accident.
--------------------------------------------------

Post ID: 2
Author: neuripsconf.bsky.social
Text: NeurIPS Conference is now Live on Bluesky! 

-NeurIPS2024 Communication Chairs
--------------------------------------------------

Post ID: 3
Author: merve.bsky.social
Text: something small but might approaches
--------------------------------------------------

Post ID: 4
Author: nicoritschel.com
Text: Metrics for when updating multiple similar queries is annoying.

If code is more your thing, that's okay too & will be supported. Note the SQL— there's no new expression language to learn.
--------------------------------------------------



# pass the data to a llm 

In [ ]:
#| echo: False
#| output: False

from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
google_api_key = os.getenv('google_api_key')
# 